In [2]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} is not supported')

        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()

            # Fix x, y = f.input, f.output
            # x.grad = f.backward(y.grad)
            
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                x.grad = gx
            
                if x.creator is not None:
                    funcs.append(x.creator)


class Function:
    def __call__(self, *inputs):
        
        # save the input variable
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
        
        self.inputs = inputs
        self.outputs = outputs
        
        # if outputs list element is not 1 then return the first element
        return outputs if len(outputs) > 1 else outputs[0]
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):
        raise NotImplementedError()

        
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy
    
class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx

    
def add(x0, x1):
    return Add()(x0, x1)
    
def square(x):
    return Square()(x)



## Problem of duplicated variable

$y = x + x$ is $y = 2x$

So, $\frac{\partial{y}}{\partial{x}} = 2$ but grad came out $1$

In [3]:
x = Variable(np.array(3.0))
y = add(x, x)
print('y', y.data)

y.backward()
print('x.grad', x.grad)

y 6.0
x.grad 1.0


## Fix Variable class

In [5]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} is not supported')

        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                
                # TODO : fix grad to be able to accumulate
                if x.grad is None:
                    x.grad = gx
                else:
                    # if we use += it will be overwrite and it could cause a problem
                    # so we use = + to copy
                    x.grad = x.grad + gx
            
                if x.creator is not None:
                    funcs.append(x.creator)

In [6]:
x = Variable(np.array(3.0))
y = add(x, x)
print('y', y.data)

y.backward()
print('x.grad', x.grad)

y 6.0
x.grad 2.0


In [7]:
x = Variable(np.array(3.0))
y = add(add(x, x), x)
print('y', y.data)

y.backward()
print('x.grad', x.grad)

y 9.0
x.grad 3.0


## Reset the grad

we can see the grad is accumulated as far as we use it

In [10]:
# first usage of x
x = Variable(np.array(2.0))
y = add(x, x)
y.backward()
print('x.grad', x.grad)

# second usage of x
y = add(add(x, x), x)
y.backward()
print('x.grad', x.grad)

x.grad 2.0
x.grad 5.0


In [11]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} is not supported')

        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            
            if not isinstance(gxs, tuple):
                gxs = (gxs,)
                
            for x, gx in zip(f.inputs, gxs):
                
                # TODO : fix grad to be able to accumulate
                if x.grad is None:
                    x.grad = gx
                else:
                    # if we use += it will be overwrite and it could cause a problem
                    # so we use = + to copy
                    x.grad = x.grad + gx
            
                if x.creator is not None:
                    funcs.append(x.creator)
    
    def clear_grad(self):
        self.grad = None

In [12]:
# first usage of x
x = Variable(np.array(2.0))
y = add(x, x)
y.backward()
print('x.grad', x.grad)

# second usage of x
# clear the original gradient
x.clear_grad()
y = add(add(x, x), x)
y.backward()
print('x.grad', x.grad)

x.grad 2.0
x.grad 3.0


## ETC

In [ ]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx
    


def exp(x):
    return Exp()(x)
